<a href="https://colab.research.google.com/github/hongli-ma/RNAsmol/blob/main/RNAsmol_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNAsmol Running Demo | [Repo](https://github.com/hongli-ma/RNAsmol)
| [Open In Colab](https://colab.research.google.com/drive/14CRDxml2PYnI1aArxOmuMtaag5AXyRTq?authuser=0#scrollTo=zngIsMg1GAd1) (click Runtime → Run all (Ctrl+F9) |

This is a code demo of RNAsmol framework for RNA-ligand interaction prediction. It takes about 5~10 minutes to run the whole pipeline.

# Setup

The first few blocks of code are necessary to set up the notebook execution environment. This checks if the notebook is running on Google Colab and installs required packages.

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

2.5.1+cu124
12.4


In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    # install packages according to the output version of the last cell
    !pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.5.1+cu124.html
    !pip install torch-geometric
    !pip install rdkit-pypi
    !pip install networkx
    !git clone https://github.com/hongli-ma/RNAsmol.git
else:
    print('Not running on CoLab')

#Import required modules

In [ ]:
%cd RNAsmol/rnasmol

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import DataLoader
import argparse
from metrics import accuracy, sensitivity, specificity,precision, f1_score, roc_auc,pr_auc,mcc_score,recall
from dataset import *
from model import MCNN_GCN
from utils import *
from metrics import *
from preprocessing import GNNDataset
import warnings
import torch.optim as optim
import random

torch.cuda.empty_cache()
warnings.filterwarnings('ignore')

#Configuration




In [ ]:
# replace the dataset name with pdb_*(rna/mol/net)perturbation
DATASET='pdb_rnaperturbation'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_root = "data"
fpath = os.path.join(data_root, DATASET)

train_set=GNNDataset(fpath,types='train')
val_set=GNNDataset(fpath, types='val')
test_set = GNNDataset(fpath, types='test')
print("Number of train: ", len(train_set))
print("Number of val: ", len(val_set))
print("Number of test: ", len(test_set))


#Data Loader





In [ ]:
# set seed before data loader
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

train_loader = DataLoader(train_set, batch_size=128, shuffle=False, num_workers=8)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=8)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=8)


model = MCNN_GCN(3, 26, embedding_size=96, filter_num=32, out_dim=2,ban_heads=2).to(device)

criterion = nn.CrossEntropyLoss()

def val(model, criterion, dataloader, device):
    model.eval()
    running_loss = AverageMeter()

    pred_list = []
    pred_cls_list = []
    label_list = []

    for data in dataloader:
        data.y = data.y.long()
        data = data.to(device)

        with torch.no_grad():
            ligand_x,protein_x,f,pred = model(data)
            loss = criterion(pred, data.y)
            pred_cls = torch.argmax(pred, dim=-1)

            pred_prob = F.softmax(pred, dim=-1)
            pred_prob, indices = torch.max(pred_prob, dim=-1)
            pred_prob[indices == 0] = 1. - pred_prob[indices == 0]

            pred_list.append(pred_prob.view(-1).detach().cpu().numpy())
            pred_cls_list.append(pred_cls.view(-1).detach().cpu().numpy())
            label_list.append(data.y.detach().cpu().numpy())
            running_loss.update(loss.item(), data.y.size(0))

    pred = np.concatenate(pred_list, axis=0)
    pred_cls = np.concatenate(pred_cls_list, axis=0)
    label = np.concatenate(label_list, axis=0)

    acc = accuracy(label, pred_cls)
    sen = sensitivity(label,pred_cls)
    spe = specificity(label,pred_cls)
    pre = precision(label, pred_cls)
    rec = recall(label, pred_cls)
    f1score=f1_score(label,pred_cls)
    rocauc = roc_auc(label, pred)
    prauc=pr_auc(label,pred)
    mcc=mcc_score(label,pred_cls)

    epoch_loss = running_loss.get_average()
    running_loss.reset()

    model.train()

    return epoch_loss, acc, sen, spe, pre, rec, f1score, rocauc, prauc, mcc, label, pred, f



#Model Training and Test
Optimize model parameters using the trainer and check test performance.

In [ ]:
# Model training on non-augmented dataset: We recommend downloading the augmented datasets for training or using our uploaded model weights from the saved_model directory.
# rerun the last cell before you train the model for more times


optimizer = optim.Adam(model.parameters(), lr=5e-4)

epochs = 20
running_loss = AverageMeter()

model.train()

for epoch in range(epochs):

    for batch_idx, data in enumerate(train_loader):

        data.y = data.y.long()
        data = data.to(device)
        ligand_x,protein_x,f,pred = model(data)

        loss = criterion(pred, data.y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss.update(loss.item(), data.y.size(0))


    epoch_loss = running_loss.get_average()
    running_loss.reset()

    val_loss, val_acc, val_sen, val_spe, val_pre, val_rec, val_f1, val_rocauc, val_prauc, val_mcc, val_label, val_pred, val_att = val(model, criterion, val_loader, device)
    test_loss, test_acc, test_sen, test_spe, test_pre, test_rec, test_f1, test_rocauc, test_prauc, test_mcc, test_label, test_pred, test_att = val(model, criterion, test_loader, device)

    msg = "epoch-%d, loss-%.4f, val_loss-%.4f, val_acc-%.4f, val_f1-%.4f, val_rocauc-%.4f, val_prauc-%.4f, val_mcc-%.4f, test_loss-%.4f, test_acc-%.4f, test_sen-%.4f, test_spe-%.4f,test_pre-%.4f, test_rec-%.4f, test_f1-%.4f, test_rocauc-%.4f, test_prauc-%.4f, test_mcc-%.4f" % (epoch, epoch_loss, val_loss, val_acc,val_f1,val_rocauc, val_prauc, val_mcc, test_loss, test_acc, test_sen, test_spe, test_pre, test_rec, test_f1, test_rocauc,test_prauc,test_mcc)
    print(msg)


# Test avoid model training

In [ ]:
# replace the model dict in saved_model folder according to DATASET name
model_path="../saved_model/pdb_bothaug_rnaperturbation.pt"

model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

test_loss, test_acc, test_sen, test_spe, test_pre, test_rec, test_f1, test_rocauc, test_prauc, test_mcc, test_label, test_pred, test_att = val(model, criterion, test_loader, device)
msg = "test_loss-%.4f, test_acc-%.4f, test_sen-%.4f, test_spe-%.4f, test_pre-%.4f, test_rec-%.4f, test_f1-%.4f, test_roauc-%.4f, test_prauc-%.4f, test_mcc-%.4f" % (test_loss, test_acc, test_sen, test_spe, test_pre, test_rec, test_f1, test_rocauc, test_prauc, test_mcc)

print(msg)


# Decoy evaluation demo

In [ ]:
from preprocessing_demo import GNNDataset

decoy_test_set = GNNDataset('data/SMN2_risdiplam_decoy', types='test')

print("Number of decoy testset: ", len(decoy_test_set))
decoy_test_loader = DataLoader(decoy_test_set, batch_size=256, shuffle=False, num_workers=8)

model = MCNN_GCN(3, 26, embedding_size=96, filter_num=32, out_dim=2,ban_heads=2).to(device)

model_path="../saved_model/robin_bothaug_molperturbation.pt"

model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

test_loss, test_acc, test_sen, test_spe, test_pre, test_rec, test_f1, test_rocauc, test_prauc, test_mcc, test_label, test_pred, test_att = val(model, criterion, decoy_test_loader, device)

folder_path = "../demo_results"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

np.save(os.path.join(folder_path, "best_model_epoch_test_pred.npy"), test_pred)

print('Note: Prediction of decoy test dataset is saved in the colab temporary directory: RNAsmol/demo_results/. You can access it by clicking Files tab on the left side of colab interface.')


#Expected Output
Awesome! You complete all demo steps and should get output like the following. Please note that these numbers might be different due to the update of environment setup on colab.


In [ ]:
#Output of 'Model Training and Test' Cell

'''
epoch-0, loss-0.6934, val_loss-0.6929, val_acc-0.5118, val_f1-0.6770, val_rocauc-0.5740, val_prauc-0.5836, val_mcc-0.0000, test_loss-0.6937, test_acc-0.4588, test_sen-1.0000, test_spe-0.0000,test_pre-0.4588, test_rec-1.0000, test_f1-0.6290, test_rocauc-0.5510, test_prauc-0.5007, test_mcc-0.0000
epoch-1, loss-0.6950, val_loss-0.6932, val_acc-0.4882, val_f1-0.0000, val_rocauc-0.5459, val_prauc-0.5516, val_mcc-0.0000, test_loss-0.6915, test_acc-0.5412, test_sen-0.0000, test_spe-1.0000,test_pre-0.0000, test_rec-0.0000, test_f1-0.0000, test_rocauc-0.5520, test_prauc-0.5158, test_mcc-0.0000
epoch-2, loss-0.6947, val_loss-0.6922, val_acc-0.5941, val_f1-0.5965, val_rocauc-0.6397, val_prauc-0.6285, val_mcc-0.1886, test_loss-0.6921, test_acc-0.5882, test_sen-0.5513, test_spe-0.6196,test_pre-0.5513, test_rec-0.5513, test_f1-0.5513, test_rocauc-0.6473, test_prauc-0.5810, test_mcc-0.1708
epoch-3, loss-0.6984, val_loss-0.6912, val_acc-0.5118, val_f1-0.6770, val_rocauc-0.7268, val_prauc-0.7271, val_mcc-0.0000, test_loss-0.6934, test_acc-0.4588, test_sen-1.0000, test_spe-0.0000,test_pre-0.4588, test_rec-1.0000, test_f1-0.6290, test_rocauc-0.7086, test_prauc-0.6784, test_mcc-0.0000
epoch-4, loss-0.6969, val_loss-0.6905, val_acc-0.5706, val_f1-0.7020, val_rocauc-0.7275, val_prauc-0.7429, val_mcc-0.2362, test_loss-0.6917, test_acc-0.5059, test_sen-0.9872, test_spe-0.0978,test_pre-0.4813, test_rec-0.9872, test_f1-0.6471, test_rocauc-0.7022, test_prauc-0.6681, test_mcc-0.1800
epoch-5, loss-0.6921, val_loss-0.6896, val_acc-0.6118, val_f1-0.4590, val_rocauc-0.7399, val_prauc-0.7560, val_mcc-0.2936, test_loss-0.6897, test_acc-0.6235, test_sen-0.3462, test_spe-0.8587,test_pre-0.6750, test_rec-0.3462, test_f1-0.4576, test_rocauc-0.6961, test_prauc-0.6645, test_mcc-0.2406
epoch-6, loss-0.6893, val_loss-0.6868, val_acc-0.6882, val_f1-0.7415, val_rocauc-0.8053, val_prauc-0.8259, val_mcc-0.3987, test_loss-0.6894, test_acc-0.6000, test_sen-0.8718, test_spe-0.3696,test_pre-0.5397, test_rec-0.8718, test_f1-0.6667, test_rocauc-0.7397, test_prauc-0.7419, test_mcc-0.2746
epoch-7, loss-0.6871, val_loss-0.6829, val_acc-0.5235, val_f1-0.6721, val_rocauc-0.8236, val_prauc-0.8582, val_mcc-0.0559, test_loss-0.6897, test_acc-0.4353, test_sen-0.9487, test_spe-0.0000,test_pre-0.4458, test_rec-0.9487, test_f1-0.6066, test_rocauc-0.7779, test_prauc-0.7949, test_mcc--0.1686
epoch-8, loss-0.6791, val_loss-0.6752, val_acc-0.6824, val_f1-0.7477, val_rocauc-0.8286, val_prauc-0.8598, val_mcc-0.4062, test_loss-0.6834, test_acc-0.5412, test_sen-0.9231, test_spe-0.2174,test_pre-0.5000, test_rec-0.9231, test_f1-0.6486, test_rocauc-0.7839, test_prauc-0.7944, test_mcc-0.1945
epoch-9, loss-0.6813, val_loss-0.6634, val_acc-0.7294, val_f1-0.7262, val_rocauc-0.8327, val_prauc-0.8640, val_mcc-0.4606, test_loss-0.6718, test_acc-0.7118, test_sen-0.7051, test_spe-0.7174,test_pre-0.6790, test_rec-0.7051, test_f1-0.6918, test_rocauc-0.7943, test_prauc-0.7997, test_mcc-0.4216
epoch-10, loss-0.6684, val_loss-0.6416, val_acc-0.7353, val_f1-0.7305, val_rocauc-0.8376, val_prauc-0.8691, val_mcc-0.4729, test_loss-0.6523, test_acc-0.7412, test_sen-0.7308, test_spe-0.7500,test_pre-0.7125, test_rec-0.7308, test_f1-0.7215, test_rocauc-0.8151, test_prauc-0.8189, test_mcc-0.4800
epoch-11, loss-0.6305, val_loss-0.6095, val_acc-0.7588, val_f1-0.6963, val_rocauc-0.8438, val_prauc-0.8643, val_mcc-0.5865, test_loss-0.6070, test_acc-0.7529, test_sen-0.5256, test_spe-0.9457,test_pre-0.8913, test_rec-0.5256, test_f1-0.6613, test_rocauc-0.8232, test_prauc-0.8213, test_mcc-0.5286
epoch-12, loss-0.5992, val_loss-0.5670, val_acc-0.7647, val_f1-0.7059, val_rocauc-0.8543, val_prauc-0.8707, val_mcc-0.5956, test_loss-0.5554, test_acc-0.7647, test_sen-0.5513, test_spe-0.9457,test_pre-0.8958, test_rec-0.5513, test_f1-0.6825, test_rocauc-0.8365, test_prauc-0.8316, test_mcc-0.5501
epoch-13, loss-0.5389, val_loss-0.5180, val_acc-0.7647, val_f1-0.7143, val_rocauc-0.8672, val_prauc-0.8822, val_mcc-0.5812, test_loss-0.5049, test_acc-0.7765, test_sen-0.5769, test_spe-0.9457,test_pre-0.9000, test_rec-0.5769, test_f1-0.7031, test_rocauc-0.8521, test_prauc-0.8457, test_mcc-0.5715
epoch-14, loss-0.4322, val_loss-0.4541, val_acc-0.7647, val_f1-0.7333, val_rocauc-0.8808, val_prauc-0.8966, val_mcc-0.5545, test_loss-0.4556, test_acc-0.7941, test_sen-0.6795, test_spe-0.8913,test_pre-0.8413, test_rec-0.6795, test_f1-0.7518, test_rocauc-0.8634, test_prauc-0.8555, test_mcc-0.5889
epoch-15, loss-0.3596, val_loss-0.4492, val_acc-0.8000, val_f1-0.7763, val_rocauc-0.8972, val_prauc-0.9082, val_mcc-0.6232, test_loss-0.4490, test_acc-0.8059, test_sen-0.6795, test_spe-0.9130,test_pre-0.8689, test_rec-0.6795, test_f1-0.7626, test_rocauc-0.8772, test_prauc-0.8665, test_mcc-0.6156
epoch-16, loss-0.2660, val_loss-0.3944, val_acc-0.8235, val_f1-0.8193, val_rocauc-0.9094, val_prauc-0.9227, val_mcc-0.6505, test_loss-0.4194, test_acc-0.8294, test_sen-0.7821, test_spe-0.8696,test_pre-0.8356, test_rec-0.7821, test_f1-0.8079, test_rocauc-0.8951, test_prauc-0.8825, test_mcc-0.6560
epoch-17, loss-0.2293, val_loss-0.4011, val_acc-0.8294, val_f1-0.8343, val_rocauc-0.9195, val_prauc-0.9351, val_mcc-0.6586, test_loss-0.4443, test_acc-0.8118, test_sen-0.8205, test_spe-0.8043,test_pre-0.7805, test_rec-0.8205, test_f1-0.8000, test_rocauc-0.9128, test_prauc-0.9014, test_mcc-0.6231
epoch-18, loss-0.1873, val_loss-1.0441, val_acc-0.6176, val_f1-0.7210, val_rocauc-0.9154, val_prauc-0.9372, val_mcc-0.3137, test_loss-1.3505, test_acc-0.5412, test_sen-0.9615, test_spe-0.1848,test_pre-0.5000, test_rec-0.9615, test_f1-0.6579, test_rocauc-0.9107, test_prauc-0.9163, test_mcc-0.2263
epoch-19, loss-0.3278, val_loss-0.3685, val_acc-0.8471, val_f1-0.8471, val_rocauc-0.9255, val_prauc-0.9388, val_mcc-0.6951, test_loss-0.4030, test_acc-0.8353, test_sen-0.8077, test_spe-0.8587,test_pre-0.8289, test_rec-0.8077, test_f1-0.8182, test_rocauc-0.9178, test_prauc-0.9067, test_mcc-0.6679
'''

#Output of 'Test avoid model training' Cell

'''
test_loss-0.1597, test_acc-0.9647, test_sen-0.9744, test_spe-0.9565, test_pre-0.9500, test_rec-0.9744, test_f1-0.9620, test_roauc-0.9915, test_prauc-0.9882, test_mcc-0.9293
'''

#Output of 'Decoy evaluation demo' Cell

'''
Number of decoy testset:  311
Done!
Note: Prediction of decoy test dataset is saved in the colab temporary directory: RNAsmol/demo_results/. You can access it by clicking Files tab on the left side of colab interface.
'''